In [7]:
from keras.models import Sequential
from keras.layers import Dense

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer

import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = None
from spellchecker import SpellChecker

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from nltk.corpus import stopwords

from keras.layers import Dense, Embedding, Dropout, LSTM, Masking
from keras.models import Sequential

from gensim.models import word2vec

import nltk

from nltk.stem import snowball

from xgboost import XGBClassifier


from sklearn.ensemble import GradientBoostingClassifier
from textdistance import levenshtein, hamming

from keras import initializers



from sklearn.naive_bayes import GaussianNB
from sklearn import svm

from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU
from keras.layers import Dense, Embedding, Bidirectional, Dropout, Flatten
from keras.optimizers import Adam, SGD
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import wordninja

from keras.layers import Embedding, Input

import lightgbm

from sklearn.decomposition import PCA

In [8]:
def unknownMap(wrd, dic):
    if wrd in dic.keys():
        return dic[wrd]
    else:
        return wrd

def tweet_processed_tokenize(wrd):
#     print(wrd)
    r=[]
    mat=re.search(r'[^a-zA-Z<>]', wrd)
    if mat:
        wrdSlice=mat.span()
#         print(wrdSlice[0])
#         print(word[wrdSlice[0]:wrdSlice[1]])
        w=wrd[:wrdSlice[0]]
        if len(w)>0:
            r.append(w)
        r.append(wrd[wrdSlice[0]:wrdSlice[1]])
        r.extend(tweet_processed_tokenize(wrd[wrdSlice[1]:]))
    else:
        if len(wrd)>0:
            r.append(wrd)
    return r
    

def handling_angulars(wrd):
    r=[]
    start=re.search('<', wrd)
    end= re.search('>', wrd)
    if start and end:
        start=start.span()[0]
        end=end.span()[1]
        if len(wrd[:start])>0:
            r.append(wrd[:start])
        r.append(wrd[start:end])
        if len(wrd[end:])>0:
            r.extend(handling_angulars(wrd[end:]))
    else:
        r.append(wrd)
    return r

def build_doc_vec(wordsList, embeddingDict, dictDimensions):
    docvec=np.zeros(dictDimensions)
    for wrd in wordsList:
#         docvec+=np.array(embedding_matrix[wrd])
        docvec+=np.array(embeddingDict[wrd])
    docvec/=len(wordsList)
    return docvec

def build_doc_vec_sif(wordsList, embeddingDict, dictDimensions):
    docvec=np.zeros(dictDimensions)
    for wrd in wordsList:
        sif=np.array(embeddingDict[wrd])*(0.0001/(0.0001+wordsList.count(wrd)))
        docvec+=sif
#     docvec/=len(wordsList)
    return docvec

def build_doc_vec_ngram(wordsList, embeddingDict, dictDimensions):
    docvec=np.zeros(dictDimensions)
    for i in range(len(wordsList)-2):
        ngram=np.zeros(dictDimensions)
        for wrd in wordsList[i:i+3]:
            ngram+=np.array(embeddingDict[wrd])
        ngram/=3
        docvec+=ngram
    return docvec

def build_doc_vec_root(wordsList, embeddingDict, dictDimensions):
    docvec=np.array(embeddingDict[wordsList[0]])
    for wrd in wordsList[1:]:
#         docvec+=np.array(embedding_matrix[wrd])
        docvec*=np.array(embeddingDict[wrd])
    docvec=np.power(docvec, 1/len(wordsList))
    return docvec

import sys
import re

FLAGS = re.MULTILINE | re.DOTALL

def hashtag(text):
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = "<hashtag> {} <allcaps>".format(hashtag_body)
    else:
        result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
    return result

def allcaps(text):
    text = text.group()
    return text.lower() + " <allcaps>"


def tokenize(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    text = re_sub(r"/"," / ")
    text = re_sub(r"@\w+", "<user>")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")
    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r"([!?.]){2,}", r"\1 <repeat>")
    text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 <elong>")

    ## -- I just don't understand why the Ruby script adds <allcaps> to everything so I limited the selection.
    # text = re_sub(r"([^a-z0-9()<>'`\-]){2,}", allcaps)
    text = re_sub(r"([A-Z]){2,}", allcaps)

    return text.lower()


def spell_contrac(a):
    if spell.correction(a) == a:
        r = spell.correction(contracting(a))
    else:
        r = spell.correction(a)
    r=lemmatizer.lemmatize(r)
    print(a, r)
    return(r)


def contracting(a):
    if len(a)>1:
        word=str(a[:2])
        for i in a[2:]:
#             print(word[-2:], i+i, word)
            if word[-2:]!=(i+i) :
                word=word+i
        return word
    else:
        return a

In [9]:
def expand(word):
    if word in contractions.keys():
        return contractions[word]
    else:
        return word

In [10]:
embeddings_dict={}
with open("glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [42]:
embeddings_dict['hi']

array([ 0.43518  ,  0.30989  , -0.094349 , -0.22371  , -0.9643   ,
       -0.10433  ,  0.090201 ,  0.1498   , -0.45896  , -0.64837  ,
       -0.036641 , -0.3369   , -0.30031  , -0.04044  , -0.16301  ,
        0.11032  ,  0.08584  , -0.10093  , -0.081372 , -0.36273  ,
        0.29575  , -0.10134  , -0.12114  , -0.0084649, -0.68382  ,
       -0.087437 ,  0.63416  , -0.021804 ,  0.11906  , -0.08871  ,
        0.62076  , -0.013319 , -0.37822  ,  0.62807  , -0.020967 ,
        0.52047  , -0.48991  ,  0.28873  , -0.38452  ,  0.30681  ,
       -0.48246  ,  0.087384 , -0.60627  ,  0.09151  , -0.26824  ,
       -0.059016 ,  0.056882 , -0.40956  ,  0.20424  , -0.090071 ,
        0.25703  ,  0.0065271, -0.22793  ,  0.03718  ,  0.62751  ,
       -0.055134 ,  0.8587   ,  0.052761 ,  0.10164  , -0.33489  ,
        0.10438  ,  0.13324  ,  0.22312  , -0.51058  , -0.17852  ,
        0.55424  , -0.36478  ,  0.55124  , -0.21034  , -0.23939  ,
        0.69448  , -0.40224  ,  0.67439  , -0.59993  , -0.4463

In [11]:
keywords_dict={}
with open("glove.6B.200d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        keywords_dict[word] = vector

In [12]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he shall",
"he'll've": "he shall have",
"he's": "he has",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has",
"i'd": "I would",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that has",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they shall",
"they'll've": "they shall have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall",
"what'll've": "what shall have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who has",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [13]:
df=pd.read_csv('train.csv')
tst=pd.read_csv('test.csv')

In [14]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [15]:
stemm=snowball.EnglishStemmer()
spell=SpellChecker()
lemmatizer=WordNetLemmatizer()

In [16]:
# df['processingtxt']=df['text'].apply(lambda x: re.sub('\'m', ' am', x))

df['retweet']=df['text'].apply(lambda x: 0 if re.subn('^RT[^:]*:', '', x)[1]==0 else 1)
df['processingtxt']=df['text'].apply(lambda x: re.subn('^RT[^:]*:', '', x)[0])

df['processingtxt']=df['text'].apply(lambda x: re.split(' |;|,|\n', x))

df['link_cnt']=df['processingtxt'].apply(lambda x: len([i for i in x if 'http' in i]))

df['tags_cnt']=df['processingtxt'].apply(lambda x: len([i for i in x if len(i)>0 and i[0]=='@']))

df['hashTags_cnt']=df['processingtxt'].apply(lambda x: len([i for i in x if len(i)>0 and i[0]=='#']))



In [17]:
df['glove_tweet_processing']=df['text'].apply(lambda x: (re.sub('[\n]', ' ',tokenize(x))).split(' '))


#Expanding Contractions
df['glove_tweet_processing1']=df['glove_tweet_processing'].apply(lambda x: [k for j in [expand(i.lower()).split(' ') for i in x] for k in j])

# df['glove_tweet_processing2']=df['glove_tweet_processing1'].apply(lambda x: [re.sub('[^a-z!\.,$=<>%-]+', '',i) for i in x])

# df['glove_tweet_processing2']=df['glove_tweet_processing1'].apply(lambda x: [i for i in x if 'http' not in i])

#Processing tweets as of glove 
df['glove_tweet_processing2']=df['glove_tweet_processing1'].apply(lambda x:[k for j in [tweet_processed_tokenize(i) for i in (x)] for k in j ])

#There were cases where in multiple tokenized tages were coming into one place. Since, there was no space between them, split couldnt take place
#Example: <url><repeat>
df['glove_tweet_processing3']=df['glove_tweet_processing2'].apply(lambda x:[k for j in [handling_angulars(i) for i in x] for k in j])

#word Ninja if multiple words got put into one. Then word Ninja.
df['glove_tweet_processing4']=df['glove_tweet_processing3'].apply(lambda x:[k.lower() for j in [wordninja.split(i) if '<' not in i else [i] for i in x] for k in j])

# df['glove_tweet_processing2']=df['glove_tweet_processing2'].apply(lambda x:[spell_contrac(i) for i in x])


In [18]:
# 5col is after mapping.. Used the same block.. Must change it back to 4 to build unknown corpus mapping
glove_corpus=[]
for i in df.glove_tweet_processing4.values:
    glove_corpus.extend(i)
glove_corpus=set(glove_corpus)

# extended_embedded_dict=embeddings_dict.copy()

train_unknown_corpus= (glove_corpus- set(embeddings_dict.keys()))
train_unknown_mapp={}
for wrd in train_unknown_corpus:
    train_unknown_mapp[wrd]=spell_contrac(wrd)
  

seawalls seagull
westmarch westmarch
angioplasty angioplasty
crematoria crematorium
underpasses underpass
mistrustful mistrustful
awans swan
mordechai mordechai
pakpattan pakpattan
shunichiro shunichiro
pyrotechnic pyrotechnic
paleface palace
officio officii
alexandrian alexandrian
globi globe
rquez quiz
suffield suffield
iiii iii
transgress transgress
bookmobile bookmobile
vigd vied
whitbourne westbourne
epiphanes epiphany
udhampur udhampur
absh ash
sizewell sizewell
bandolier bandolier
osco sco
coiffed coffee
menahem menachem
inundation inundation
flimflam flimflam
waimate climate
marinading marinading
transwomen townswomen
ekstr east
okinawan okinawa
teslas tessas
nurgle nurgle
wahhabism wahhabism
pulwama pulham
antiochus antiochus
desecrates desecrated
cummerbund cummerbund
tawee twee
skardu shard
steamship steamship
pilloried pilloried
vestment vestment
policyholders policyholder
diyala diana
iinet ninet
bigamist bigamist
galv gall
hisao hsiao
catechize catechism
trajectories traj

In [12]:
len(glove_corpus)

12386

In [13]:
len(train_unknown_corpus)

127

In [20]:
df['glove_tweet_processing5']=df['glove_tweet_processing4'].apply(lambda x:[unknownMap(i, train_unknown_mapp) for i in x])


In [22]:
# 5col is after mapping.. Used the same block.. Must change it back to 4 to build unknown corpus mapping
glove_corpus=[]
for i in df.glove_tweet_processing5.values:
    glove_corpus.extend(i)
glove_corpus=set(glove_corpus)


In [23]:
len(glove_corpus- set(embeddings_dict.keys()))

67

In [24]:
df.head(10)

,id,keyword,location,text,target,retweet,processingtxt,link_cnt,tags_cnt,hashTags_cnt,glove_tweet_processing,glove_tweet_processing1,glove_tweet_processing2,glove_tweet_processing3,glove_tweet_processing4,glove_tweet_processing5
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,0,"[Our, Deeds, are, the, Reason, of, this, #earthquake, May, ALLAH, Forgive, us, all]",0,0,1,"[our, deeds, are, the, reason, of, this, <hashtag>, earthquake, may, allah, <allcaps>, forgive, us, all]","[our, deeds, are, the, reason, of, this, <hashtag>, earthquake, may, allah, <allcaps>, forgive, us, all]","[our, deeds, are, the, reason, of, this, <hashtag>, earthquake, may, allah, <allcaps>, forgive, us, all]","[our, deeds, are, the, reason, of, this, <hashtag>, earthquake, may, allah, <allcaps>, forgive, us, all]","[our, deeds, are, the, reason, of, this, <hashtag>, earthquake, may, allah, <allcaps>, forgive, us, all]","[our, deeds, are, the, reason, of, this, <hashtag>, earthquake, may, allah, <allcaps>, forgive, us, all]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0,"[Forest, fire, near, La, Ronge, Sask., Canada]",0,0,0,"[forest, fire, near, la, ronge, sask., canada]","[forest, fire, near, la, ronge, sask., canada]","[forest, fire, near, la, ronge, sask, ., canada]","[forest, fire, near, la, ronge, sask, ., canada]","[forest, fire, near, la, rong, e, s, ask, canada]","[forest, fire, near, la, rong, e, s, ask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,0,"[All, residents, asked, to, 'shelter, in, place', are, being, notified, by, officers., No, other, evacuation, or, shelter, in, place, orders, are, expected]",0,0,0,"[all, residents, asked, to, 'shelter, in, place', are, being, notified, by, officers., no, other, evacuation, or, shelter, in, place, orders, are, expected]","[all, residents, asked, to, 'shelter, in, place', are, being, notified, by, officers., no, other, evacuation, or, shelter, in, place, orders, are, expected]","[all, residents, asked, to, ', shelter, in, place, ', are, being, notified, by, officers, ., no, other, evacuation, or, shelter, in, place, orders, are, expected]","[all, residents, asked, to, ', shelter, in, place, ', are, being, notified, by, officers, ., no, other, evacuation, or, shelter, in, place, orders, are, expected]","[all, residents, asked, to, ', shelter, in, place, ', are, being, notified, by, officers, no, other, evacuation, or, shelter, in, place, orders, are, expected]","[all, residents, asked, to, ', shelter, in, place, ', are, being, notified, by, officers, no, other, evacuation, or, shelter, in, place, orders, are, expected]"
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1,0,"[13, 000, people, receive, #wildfires, evacuation, orders, in, California, ]",0,0,1,"[<number>, people, receive, <hashtag>, wildfires, evacuation, orders, in, california, ]","[<number>, people, receive, <hashtag>, wildfires, evacuation, orders, in, california, ]","[<number>, people, receive, <hashtag>, wildfires, evacuation, orders, in, california]","[<number>, people, receive, <hashtag>, wildfires, evacuation, orders, in, california]","[<number>, people, receive, <hashtag>, wildfires, evacuation, orders, in, california]","[<number>, people, receive, <hashtag>, wildfires, evacuation, orders, in, california]"
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1,0,"[Just, got, sent, this, photo, from, Ruby, #Alaska, as, smoke, from, #wildfires, pours, into, a, school, ]",0,0,2,"[just, got, sent, this, photo, from, ruby, <hashtag>, , alaska, as, smoke, from, <hashtag>, wildfires, pours, into, a, school, ]","[just, got, sent, this, photo, from, ruby, <hashtag>, , alaska, as, smoke, from, <hashtag>, wildfires, pours, into, a, school, ]","[just, got, sent, this, photo, from, ruby, <hashtag>, alaska, as, sm

In [34]:
df['modelling']=df['glove_tweet_processing5'].apply(lambda x:([i for i in x if i in embeddings_dict.keys()]))

#Removing Special chars other than <>
df['modelling']=df['modelling'].apply(lambda x:[re.sub('[^a-z<>]+', '',i) for i in x])

df['modelling']=df['modelling'].apply(lambda x:[i for i in x if len(i)>0 ]) # i not in ['.','\'',','] and

df['docVec']=df['modelling'].apply(lambda x: build_doc_vec(x, embeddings_dict, 200))

df[['txtVec_D'+str(i) for i in range(200)]]=pd.DataFrame(df.docVec.tolist())

('hi' + 'name')/2

In [25]:
embeddings_dict['hi']

array([ 0.43518  ,  0.30989  , -0.094349 , -0.22371  , -0.9643   ,
       -0.10433  ,  0.090201 ,  0.1498   , -0.45896  , -0.64837  ,
       -0.036641 , -0.3369   , -0.30031  , -0.04044  , -0.16301  ,
        0.11032  ,  0.08584  , -0.10093  , -0.081372 , -0.36273  ,
        0.29575  , -0.10134  , -0.12114  , -0.0084649, -0.68382  ,
       -0.087437 ,  0.63416  , -0.021804 ,  0.11906  , -0.08871  ,
        0.62076  , -0.013319 , -0.37822  ,  0.62807  , -0.020967 ,
        0.52047  , -0.48991  ,  0.28873  , -0.38452  ,  0.30681  ,
       -0.48246  ,  0.087384 , -0.60627  ,  0.09151  , -0.26824  ,
       -0.059016 ,  0.056882 , -0.40956  ,  0.20424  , -0.090071 ,
        0.25703  ,  0.0065271, -0.22793  ,  0.03718  ,  0.62751  ,
       -0.055134 ,  0.8587   ,  0.052761 ,  0.10164  , -0.33489  ,
        0.10438  ,  0.13324  ,  0.22312  , -0.51058  , -0.17852  ,
        0.55424  , -0.36478  ,  0.55124  , -0.21034  , -0.23939  ,
        0.69448  , -0.40224  ,  0.67439  , -0.59993  , -0.4463

In [29]:
keywordMapp={
'unknown':'unknown',
'ablaz':'ablaze',
'accid':'accident'
,'aftershock':'aftershock'
,'airplane accid':'plane accident'
,'ambul':'ambulance'
,'annihil':'annihilate'
,'apocalyps':'armageddon'
,'armageddon':'armageddon'
,'armi':'army'
,'arson':'arson'
,'arsonist':'arsonist'
,'attack':'attack'
,'avalanch':'avalanche'
,'battl':'army'
,'bioterror':'bio terror'
,'blaze':'blaze'
,'bleed':'blood'
,'blew up':'blowup'
,'blight':'blight'
,'blizzard':'blizzard'
,'blood':'blood'
,'bloodi':'blood'
,'blown up':'blow up'
,'body bag':'body bag'
,'bomb':'bomb'
,'bridge collaps':'bridge collapse'
,'buildings burn':'buildings burn'
,'buildings on fir':'buildings burn'
,'burn':'burn'
,'burning build':'buildings burn'
,'bush fir':'bush fire'
,'casualti':'casualty'
,'catastroph':'catastrophe'
,'chemical emerg':'emergency'
,'cliff fal':'cliff fall'
,'collaps':'collapse'
,'collid':'collide'
,'collis':'collision'
,'crash':'crash'
,'crush':'crush'
,'curfew':'curfew'
,'cyclon':'cyclone'
,'damag':'damage'
,'danger':'danger'
,'dead':'dead'
,'death':'death'
,'debri':'debris'
,'delug':'deluge'
,'demolish':'demolish'
,'demolit':'demolish'
,'derail':'derail'
,'desol':'desolation'
,'destroy':'destroy'
,'destruct':'destroy'
,'deton':'detonate'
,'devast':'devastation'
,'disast':'disaster'
,'displac':'evacuation'
,'drought':'drought'
,'drown':'drown'
,'dust storm':'dust storm'
,'earthquak':'earthquake'
,'electrocut':'electrocute'
,'emerg':'emergency'
,'emergency plan':'emergency'
,'emergency servic':'emergency'
,'engulf':'engulf'
,'epicentr':'aftershock'
,'evacu':'evacuation'
,'explod':'explode'
,'explos':'explosion'
,'eyewit':'eyewitness'
,'famin':'famine'
,'fatal':'fatal'
,'fear':'fear'
,'fire':'fire'
,'fire truck':'fire truck'
,'first respond':'response'
,'flame':'fire'
,'flatten':'flatten'
,'flood':'flood'
,'forest fir':'forest fire'
,'hail':'hail'
,'hailstorm':'hail storm'
,'harm':'harm'
,'hazard':'hazard'
,'heat wav':'heatwave'
,'hellfir':'hell fire'
,'hijack':'hijack'
,'hostag':'hostage'
,'hurrican':'hurricane'
,'injur':'injured'
,'injuri':'injury'
,'inund':'deluge'
,'landslid':'landslide'
,'lava':'lava'
,'lightn':'lightning'
,'loud bang':'noise'
,'mass murder':'murderer'
,'mass murd':'murder'
,'massacr':'massacre'
,'mayhem':'mayhem'
,'meltdown':'meltdown'
,'militari':'army'
,'mudslid':'mudslide'
,'natural disast':'natural disaster'
,'nuclear disast':'nuclear disaster'
,'nuclear reactor':'nuclear reactor'
,'obliter':'obliterate'
,'oil spil':'oil'
,'outbreak':'outbreak'
,'pandemonium':'panic'
,'panic':'panic'
,'panick':'panic'
,'polic':'police'
,'quarantin':'quarantine'
,'radiation emerg':'emergency'
,'rainstorm':'rain'
,'raze':'raze'
,'refug':'refuge'
,'refuge':'refuge'
,'rescu':'rescued'
,'rescuer':'rescuer'
,'riot':'riot'
,'rubbl':'rubble'
,'ruin':'ruin'
,'sandstorm':'sandstorm'
,'scream':'scream'
,'seismic':'seismic'
,'sinkhol':'sink hole'
,'sink':'sink'
,'siren':'siren'
,'smoke':'smoke'
,'snowstorm':'snowstorm'
,'storm':'storm'
,'stretcher':'stretcher'
,'structural failur':'collapse'
,'suicide bomb':'suicide bomb'
,'sunk':'drown'
,'surviv':'survived'
,'survivor':'refuge'
,'terror':'terror'
,'terrorist':'terror'
,'threat':'threat'
,'thunder':'thunder'
,'thunderstorm':'thunderstorm'
,'tornado':'tornado'
,'tragedi':'tragedy'
,'trap':'trap'
,'trauma':'trauma'
,'traumatis':'traumatised'
,'troubl':'trouble'
,'tsunami':'tsunami'
,'twister':'twister'
,'typhoon':'typhoon'
,'upheav':'upheaval'
,'violent storm':'violent storm'
,'volcano':'volcano'
,'war zon':'army'
,'weapon':'weapon'
,'whirlwind':'whirlwind'
,'wild fir':'wildfire'
,'wildfir':'wildfire'
,'windstorm':'windstorm'
,'wound':'wound'
,'wreck':'wrecked'
,'wreckag':'wreck'
}

In [30]:
df['keyword']=df['keyword'].fillna('unknown', axis=0)

df['keyword1']=df['keyword'].apply(lambda x: stemm.stem(re.sub(r'[^a-zA-Z]+', ' ', x)))
df['keyword1']=df['keyword1'].apply(lambda x: keywordMapp[x])

tp=pd.get_dummies(df[['keyword1']]).drop('keyword1_unknown', axis=1)
pca=PCA(n_components=115)
tp=pd.DataFrame(pca.fit_transform(tp))
df[tp.columns]=tp[tp.columns]

In [31]:
pd.DataFrame(tp).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114
0,-0.002232,-0.00763,-0.01276,-0.003631,-0.005215,-0.003743,-0.004431,-0.002972,2.485261e-15,-0.003376,-4.075729e-14,4.239487e-15,0.002273,0.002911,0.005472,0.003307,0.003218,0.005555,0.01598,-0.015088,-0.002499,-0.002804,-0.003256,-1.850096e-14,9.540810e-15,-0.004335,-0.001774,-5.574945e-15,2.321186e-14,6.121342e-15,0.003045,-0.002367,1.313083e-15,1.785902e-15,1.640544e-14,0.002584,-3.214131e-14,7.708551e-15,-0.002607,-2.087311e-14,-1.205263e-14,8.312428e-15,0.002487,-8.015371e-15,1.091764e-14,0.002503,5.110057e-15,1.245260e-14,0.001872,0.002278,0.009703,0.023347,-0.013882,-0.005448,-6.373014e-16,6.565315e-15,-0.002448,-2.261642e-15,7.343950e-16,5.672190e-15,-0.002469,-3.358337e-15,-3.827556e-15,-8.632822e-15,1.799012e-15,2.726110e-16,1.114345e-14,4.476880e-15,-0.003095,-4.616608e-15,-7.725006e-15,2.610021e-15,6.969858e-15,1.686240e-14,-0.003195,-1.345632e-14,-3.383206e-15,-6.514486e-15,1.233914e-15,8.552324e-15,9.914180e-15,-0.003139,-7.879765e-15,1.544338e-15,-6.377060e-15,-2.656729e-15,-1.188015e-15,-4.898832e-16,5.991301e-15,-0.002927,-1.236663e-14,-2.494813e-15,8.793905e-16,1.180218e-16,2.873573e-16,4.489959e-18,1.285411e-15,3.769000e-15,-3.204548e-15,8.595064e-17,3.116031e-15,0.002797,-1.070303e-14,-2.067271e-15,-2.155146e-15,-1.804911e-16,-3.717295e-16,1.703128e-15,3.317433e-16,2.606532e-15,1.876349e-15,0.002771,-1.207661e-15,5.391344e-17,-5.635025e-16
1,-0.002232,-0.00763,-0.01276,-0.003631,-0.005215,-0.003743,-0.004431,-0.002972,-3.095361e-15,-0.003376,-3.863755e-14,1.131419e-14,0.002273,0.002911,0.005472,0.003307,0.003218,0.005555,0.01598,-0.015088,-0.002499,-0.002804,-0.003256,-1.926306e-14,9.097155e-15,-0.004335,-0.001774,-1.810572e-15,2.194749e-14,1.699231e-16,0.003045,-0.002367,-1.286065e-15,7.274194e-15,2.299067e-14,0.002584,-2.921060e-14,9.376859e-15,-0.002607,-2.044355e-14,-1.244996e-14,6.810954e-15,0.002487,-9.012448e-15,1.352572e-14,0.002503,-7.045341e-16,5.809347e-15,0.001872,0.002278,0.009703,0.023347,-0.013882,-0.005448,-1.850812e-15,2.995247e-15,-0.002448,-3.211693e-15,1.178049e-15,6.001331e-15,-0.002469,-3.175524e-15,-4.094327e-15,-5.847298e-15,-1.034415e-15,-1.299155e-15,1.267772e-14,7.607716e-15,-0.003095,-7.094672e-15,-9.750934e-15,1.630057e-15,9.004473e-15,1.768847e-14,-0.003195,-1.047673e-14,-6.632536e-15,-6.432709e-15,3.519062e-15,6.095048e-15,8.178390e-15,-0.003139,-8.681207e-15,-1.929446e-15,-6.471386e-15,-6.636618e-15,-2.371313e-15,1.684281e-15,2.218278e-15,-0.002927,-6.969699e-15,-1.369437e-15,9.852252e-16,-2.152294e-15,2.373264e-17,1.422034e-15,1.154561e-16,6.210896e-15,2.291162e-15,3.027515e-15,3.633018e-15,0.002797,-9.245191e-15,-2.205089e-15,-2.457018e-15,-2.241757e-15,9.482896e-17,4.987371e-15,2.819581e-16,1.247949e-15,9.211053e-16,0.002771,-2.535216e-15,-3.890818e-15,-5.407693e-16
2,-0.002232,-0.00763,-0.01276,-0.003631,-0.005215,-0.003743,-0.004431,-0.002972,-4.248270e-15,-0.003376,-3.669556e-14,9.278877e-15,0.002273,0.002911,0.005472,0.003307,0.003218,0.005555,0.01598,-0.015088,-0.002499,-0.002804,-0.003256,-1.869545e-14,1.144608e-14,-0.004335,-0.001774,-1.511879e-15,1.606413e-14,4.315681e-15,0.003045,-0.002367,-4.268872e-16,3.264697e-15,2.423170e-14,0.002584,-2.890125e-14,1.138670e-14,-0.002607,-1.992892e-14,-1.252356e-14,6.340492e-15,0.002487,-1.501957e-14,1.537498e-14,0.002503,5.402011e-16,6.220179e-15,0.001872,0.002278,0.009703,0.023347,-0.013882,-0.005448,-3.185119e-15,3.228855e-15,-0.002448,-2.674439e-15,-2.242271e-16,8.516514e-15,-0.002469,-4.704380e-15,-4.816099e-15,-7.958446e-15,-1.727918e-15,-2.314781e-16,1.196407e-14,7.157455e-15,-0.003095,-6.311954e-15,-8.137711e-15,5.982542e-16,7.783069e-15,1.843301e-14,-0.003195,-9.732819e-15,

In [564]:
# df.drop([i for i in range(120)], axis=1, inplace=True)

In [35]:
tmp=df.copy()

In [36]:
x=tmp.drop(['id', 'keyword', 'location', 'text', 'target', 'processingtxt','keyword1',
            'glove_tweet_processing', 'docVec',
           'glove_tweet_processing1', 'glove_tweet_processing2' ,
            'glove_tweet_processing3', 'glove_tweet_processing4' ,'glove_tweet_processing5' ,
            'modelling',
           ], axis=1)

In [37]:
y=tmp[['target']].copy()

In [38]:
x.head(1)

,retweet,link_cnt,tags_cnt,hashTags_cnt,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,txtVec_D0,txtVec_D1,txtVec_D2,txtVec_D3,txtVec_D4,txtVec_D5,txtVec_D6,txtVec_D7,txtVec_D8,txtVec_D9,txtVec_D10,txtVec_D11,txtVec_D12,txtVec_D13,txtVec_D14,txtVec_D15,txtVec_D16,txtVec_D17,txtVec_D18,txtVec_D19,txtVec_D20,txtVec_D21,txtVec_D22,txtVec_D23,txtVec_D24,txtVec_D25,txtVec_D26,txtVec_D27,txtVec_D28,txtVec_D29,txtVec_D30,txtVec_D31,txtVec_D32,txtVec_D33,txtVec_D34,txtVec_D35,txtVec_D36,txtVec_D37,txtVec_D38,txtVec_D39,txtVec_D40,txtVec_D41,txtVec_D42,txtVec_D43,txtVec_D44,txtVec_D45,txtVec_D46,txtVec_D47,txtVec_D48,txtVec_D49,txtVec_D50,txtVec_D51,txtVec_D52,txtVec_D53,txtVec_D54,txtVec_D55,txtVec_D56,txtVec_D57,txtVec_D58,txtVec_D59,txtVec_D60,txtVec_D61,txtVec_D62,txtVec_D63,txtVec_D64,txtVec_D65,txtVec_D66,txtVec_D67,txtVec_D68,txtVec_D69,txtVec_D70,txtVec_D71,txtVec_D72,txtVec_D73,txtVec_D74,txtVec_D75,txtVec_D76,txtVec_D77,txtVec_D78,txtVec_D79,txtVec_D80,txtVec_D81,txtVec_D82,txtVec_D83,txtVec_D84,txtVec_D85,txtVec_D86,txtVec_D87,txtVec_D88,txtVec_D89,txtVec_D90,txtVec_D91,txtVec_D92,txtVec_D93,txtVec_D94,txtVec_D95,txtVec_D96,txtVec_D97,txtVec_D98,txtVec_D99,txtVec_D100,txtVec_D101,txtVec_D102,txtVec_D103,txtVec_D104,txtVec_D105,txtVec_D106,txtVec_D107,txtVec_D108,txtVec_D109,txtVec_D110,txtVec_D111,txtVec_D112,txtVec_D113,txtVec_D114,txtVec_D115,txtVec_D116,txtVec_D117,txtVec_D118,txtVec_D119,txtVec_D120,txtVec_D121,txtVec_D122,txtVec_D123,txtVec_D124,txtVec_D125,txtVec_D126,txtVec_D127,txtVec_D128,txtVec_D129,txtVec_D130,txtVec_D131,txtVec_D132,txtVec_D133,txtVec_D134,txtVec_D135,txtVec_D136,txtVec_D137,txtVec_D138,txtVec_D139,txtVec_D140,txtVec_D141,txtVec_D142,txtVec_D143,txtVec_D144,txtVec_D145,txtVec_D146,txtVec_D147,txtVec_D148,txtVec_D149,txtVec_D150,txtVec_D151,txtVec_D152,txtVec_D153,txtVec_D154,txtVec_D155,txtVec_D156,txtVec_D157,txtVec_D158,txtVec_D159,txtVec_D160,txtVec_D161,txtVec_D162,txtVec_D163,txtVec_D164,txtVec_D165,txtVec_D166,txtVec_D167,txtVec_D168,txtVec_D169,txtVec_D170,txtVec_D171,txtVec_D172,txtVec_D173,txtVec_D174,txtVec_D175,txtVec_D176,txtVec_D177,txtVec_D178,txtVec_D179,txtVec_D180,txtVec_D181,txtVec_D182,txtVec_D183,txtVec_D184,txtVec_D185,txtVec_D186,txtVec_D187,txtVec_D188,txtVec_D189,txtVec_D190,txtVec_D191,txtVec_D192,txtVec_D193,txtVec_D194,txtVec_D195,txtVec_D196,txtVec_D197,txtVec_D198,txtVec_D199
0,0,0,0,1,-0.002232,-0.00763,-0.01276,-0.003631,-0.005215,-0.003743,-0.004431,-0.002972,2.485261e-15,-0.003376,-4.075729e-14,4.239487e-15,0.002273,0.002911,0.005472,0.003307,0.003218,0.005555,0.01598,-0.015088,-0.002499,-0.002804,-0.003256,-1.850096e-14,9.540810e-15,-0.004335,-0.001774,-5.574945e-15,2.321186e-14,6.121342e-15,0.003045,-0.002367,1.313083e-15,1.785902e-15,1.640544e-14,0.002584,-3.214131e-14,7.708551e-15,-0.002607,-2.087311e-14,-1.205263e-14,8.312428e-15,0.002487,-8.015371e-15,1.091764e-14,0.002503,5.110057e-15,1.245260e-14,0.001872,0.002278,0.009703,0.023347,-0.013882,-0.005448,-6.373014e-16,6.565315e-15,-0.002448,-2.261642e-15,7.343950e-16,5.672190e-15,-0.002469,-3.358337e-15,-3.827556e-15,-8.632822e-15,1.799012e-15,2.726110e-16,1.114345e-14,4.476880e-15,-0.003095,-4.616608e-15,-7.725006e-15,2.610021e-15,6.969858e-15,1.686240e-14,-0.003195,-1.345632e-14,-3.383206e-15,-6.514486e-15,1.233914e-15,8.552324e-15,9.914180e-15,-0.003139,-7.879765e-15,1.544338e-15,-6.377060e-15,-2.656729e-15,-1.188015e-15,-4.898832e-16,5.991301e-15,-0.002927,-1.236663e-14,-2.494813e-15,8.793905e-16,1.180218e-16,2.873573e-16,4.489959e-18,1.285411e-15,3.769000e-15,-3.204548e-15,8.595064e-17,3.116031e-15,0.002797,-1.070303e-14,-2.067271e-15,-2.155146e-15,-1.804911e-16,-3.717295e-16,1.703128e-15,3.317433e-16,2.606

In [39]:
x.isna().sum()

retweet         0
link_cnt        0
tags_cnt        0
hashTags_cnt    0
0               0
1               0
2               0
3               0
4               0
5               0
6               0
7               0
8               0
9               0
10              0
11              0
12              0
13              0
14              0
15              0
16              0
17              0
18              0
19              0
20              0
21              0
22              0
23              0
24              0
25              0
26              0
27              0
28              0
29              0
30              0
31              0
32              0
33              0
34              0
35              0
36              0
37              0
38              0
39              0
40              0
41              0
42              0
43              0
44              0
45              0
46              0
47              0
48              0
49              0
50              0
51        

In [40]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [41]:
clf = LogisticRegression(solver='saga', max_iter=250, random_state=1)

clf.fit(X_train, y_train)

y_pred=clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
# logistic=LogisticRegression(solver='saga', max_iter=200, random_state=1)
# logistic.fit(x, y)

C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8253064798598949
0.7779632721202003


C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [36]:
gb = GradientBoostingClassifier(n_estimators=400, random_state=1)
classifier(gb)

C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8152364273204904
0.7639821029082774


In [35]:
rf= RandomForestClassifier(n_estimators=150)
classifier(rf)

C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.8003502626970228
0.7311320754716981


In [37]:
gnb=GaussianNB()
classifier(gnb)

0.7504378283712785
0.7079918032786885


C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [31]:
sv=svm.SVC()
classifier(sv)

C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7902802101576182
0.7109233554616777


In [60]:
xgb= XGBClassifier(n_estimators=1000, learning_rate=0.07, max_depth=100 )
classifier(xgb)

0.8218038528896673
0.7686185332575326


In [61]:
# xgb= XGBClassifier(n_estimators=800, learning_rate=0.07, max_depth=100 ) V2. Best Model
#xgb= XGBClassifier(n_estimators=1000, learning_rate=0.07, max_depth=100 ) #0.8218038528896673 V3 Best model Kaggle Score: 82.347
xgb= XGBClassifier(n_estimators=1000, learning_rate=0.07, max_depth=100 )
xgb.fit(x,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.07, max_delta_step=0,
              max_depth=100, min_child_weight=1, missing=None,
              n_estimators=1000, n_jobs=1, nthread=None,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
              subsample=1)

In [42]:
lgbm = lightgbm.LGBMClassifier(random_state=1, num_iterations=500, learning_rate=0.08)
classifier(lgbm)

C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.8147985989492119
0.760068065796937


In [627]:
# lgbm=lightgbm.LGBMClassifier(random_state=1, num_iterations=500, learning_rate=0.08) Best Params
lgbm=lightgbm.LGBMClassifier(random_state=1, num_iterations=500, learning_rate=0.08) 
lgbm.fit(x,y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_iterations=400, num_leaves=31,
               objective=None, random_state=1, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [30]:
def classifier(model):
    model.fit(X_train, y_train)
    model_pred=model.predict(X_test)
    acc_score=accuracy_score(y_test, model_pred)
    print(acc_score)
    print(f1_score(y_test, model_pred))

In [108]:
np.shape(X_train.values)

(5329, 319)

In [ ]:
model = Sequential()
lstm.add(Embedding(input_dim = 188, output_dim = 50, input_length = input_length))
model.add(LSTM(output_dim=256, activation='sigmoid', inner_activation='hard_sigmoid', return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(output_dim=256, activation='sigmoid', inner_activation='hard_sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [109]:
lstm=Sequential()
lstm.add(Embedding(input_dim = 188, output_dim = 50, input_length = x.shape[1]))
lstm.add(LSTM(units=10, input_shape=np.shape(X_train.values), return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=10, return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=10, return_sequences=True))
lstm.add(Dropout(0.2))
lstm.add(LSTM(units=1))
lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [110]:
lstm.fit(np.reshape(X_train.values, (1, X_train.shape[0], X_train.shape[1])), y_train.target.values, batch_size=10, nb_epoch=50)

C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


ValueError: Input arrays should have the same number of samples as target arrays. Found 1 input samples and 5329 target samples.

In [103]:
np.reshape(pd.get_dummies(y_train.target).values, ( 1,2))

ValueError: cannot reshape array of size 10658 into shape (1,2)

In [71]:
np.reshape(X_train.values, (1, X_train.shape[0], X_train.shape[1]))

array([[[ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
          7.02582711e-16,  2.39495973e-16, -2.39913066e-17],
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
         -6.29038169e-15, -2.29642134e-15,  1.26174559e-16],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
          2.68995751e-14,  1.47305820e-14, -3.76822628e-14],
        ...,
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
          5.85138010e-16,  1.38073911e-16,  3.24164878e-17],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          1.72473232e-15,  6.58631354e-16,  1.23220573e-16],
        [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
          6.28432171e-16,  2.46882677e-16, -3.30720994e-16]]])

In [35]:
ann=Sequential()
ann.add(Dense(output_dim=30, init='glorot_uniform', activation='relu', input_dim=x.shape[1]))
ann.add(Dense(output_dim=30, init='glorot_uniform', activation='tanh'))
ann.add(Dense(output_dim=30, init='glorot_uniform', activation='relu'))
ann.add(Dense(output_dim=1, init='glorot_uniform', activation='sigmoid'))
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=344, units=30, kernel_initializer="glorot_uniform")`
  
C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", units=30, kernel_initializer="glorot_uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=30, kernel_initializer="glorot_uniform")`
  after removing the cwd from sys.path.
C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="glorot_uniform")`
  """


In [36]:
ann.fit(X_train, y_train, batch_size=10, nb_epoch=150)

C:\Users\SirajuddinMohammed\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/150
6090/6090 [==============================] - 2s 307us/step - loss: 0.4800 - accuracy: 0.7806
Epoch 2/150
6090/6090 [==============================] - 1s 221us/step - loss: 0.4148 - accuracy: 0.8154
Epoch 3/150
6090/6090 [==============================] - 1s 225us/step - loss: 0.3918 - accuracy: 0.8333
Epoch 4/150
6090/6090 [==============================] - 1s 206us/step - loss: 0.3794 - accuracy: 0.8356
Epoch 5/150
6090/6090 [==============================] - 1s 220us/step - loss: 0.3583 - accuracy: 0.8499
Epoch 6/150
6090/6090 [==============================] - 1s 219us/step - loss: 0.3410 - accuracy: 0.8581
Epoch 7/150
6090/6090 [==============================] - 1s 233us/step - loss: 0.3292 - accuracy: 0.8640
Epoch 8/150
6090/6090 [==============================] - 1s 221us/step - loss: 0.3111 - accuracy: 0.8708
Epoch 9/150
6090/6090 [==============================] - 1s 224us/step - loss: 0.2886 - accuracy: 0.8790
Epoch 10/150
6090/6090 [==============================]

In [37]:
ann_ypred=ann.predict(X_test)

In [38]:
accuracy_score(y_test==1,ann_ypred>=0.5)

0.7655942219304005

In [39]:
ann_e=Sequential()
ann_e.add(Dense(output_dim=10, init='uniform', activation='sigmoid', input_dim=x.shape[1]))
ann_e.add(Dense(output_dim=10, init='uniform', activation='sigmoid'))
ann_e.add(Dense(output_dim=10, init='uniform', activation='sigmoid'))
ann_e.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))
ann_e.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ann_e.fit(x, y, batch_size=10, nb_epoch=150)

In [44]:
tst['processingtxt']=tst['text'].apply(lambda x: re.sub('\'m', ' am', x))

tst['retweet']=tst['processingtxt'].apply(lambda x: 0 if re.subn('^RT[^:]*:', '', x)[1]==0 else 1)
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.subn('^RT[^:]*:', '', x)[0])

tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.split(' |;|,|\n', x))

tst['link_cnt']=tst['processingtxt'].apply(lambda x: len([i for i in x if 'http' in i]))

tst['tags_cnt']=tst['processingtxt'].apply(lambda x: len([i for i in x if len(i)>0 and i[0]=='@']))

tst['hashTags_cnt']=tst['processingtxt'].apply(lambda x: len([i for i in x if len(i)>0 and i[0]=='#']))



tst['glove_tweet_processing']=tst['text'].apply(lambda x: (re.sub('[\n]', ' ',tokenize(x))).split(' '))

tst['glove_tweet_processing1']=tst['glove_tweet_processing'].apply(lambda x: [k for j in [expand(i.lower()).split(' ') for i in x] for k in j])

# df['glove_tweet_processing2']=df['glove_tweet_processing1'].apply(lambda x: [re.sub('[^a-z!\.,$=<>%-]+', '',i) for i in x])

# df['glove_tweet_processing2']=df['glove_tweet_processing1'].apply(lambda x: [i for i in x if 'http' not in i])


tst['glove_tweet_processing2']=tst['glove_tweet_processing1'].apply(lambda x:[k for j in [tweet_processed_tokenize(i) for i in (x)] for k in j ])

tst['glove_tweet_processing3']=tst['glove_tweet_processing2'].apply(lambda x:[k for j in [handling_angulars(i) for i in x] for k in j])

tst['glove_tweet_processing4']=tst['glove_tweet_processing3'].apply(lambda x:[k.lower() for j in [wordninja.split(i) if '<' not in i else [i] for i in x] for k in j])


In [45]:
# 5 col is after mapping.. Used the same block.. Must change it back to 4 to build unknown corpus mapping
tst_glove_corpus=[]
for i in tst.glove_tweet_processing4.values:
    tst_glove_corpus.extend(i)
tst_glove_corpus=set(tst_glove_corpus)

tst_unknown_corpus= (tst_glove_corpus- set(embeddings_dict.keys()))
tst_unknown_mapp={}
for wrd in tst_unknown_corpus:
    tst_unknown_mapp[wrd]=spell_contrac(wrd)

referential referential
toothsome toothsome
underskirt underskirt
statesmanlike statesmanlike
pontin pontin
netnews nephew
katayama nakayama
oriya erika
crawfordsville crawfordsville
clambake clambake
perturbs perturb
permut permit
uttara uttar
neelum vellum
vestment vestment
waimate climate
isberg iceberg
fishiest fish
corrodes corrodes
chail chair
ulcerated ulcerated
netbot nebot
yaound found
zeroth zero
aphid aphid
crematoria crematorium
ibrahimovi ibrahimovi
sharecropping sharecropping
timbering limbering
maghar magyar
ramify ratify
rifted lifted
stanis stand
appropriates appropriates
sauntered sauntered
toshikazu toshikazu
jalape palace
bigamist bigamist
maimonides simonides
autocracies autocracy
fishtails fishtails
charnock charnock
stampedes stampede
retrofitting retrofitting
presqu press
acomb acomb
udhampur udhampur
torpedoed torpedoed
inundation inundation
pyrotechnic pyrotechnic
manorial manorial
canaanites canaanite
hussars hussar
salivation salivation
nberg berg
occitan oc

In [46]:
len(tst_glove_corpus)

8138

In [47]:
len(tst_unknown_corpus)

65

In [48]:
tst['glove_tweet_processing5']=tst['glove_tweet_processing4'].apply(lambda x:[unknownMap(i, tst_unknown_mapp) for i in x])

In [49]:
tst['modelling']=tst['glove_tweet_processing5'].apply(lambda x:([i for i in x if i in embeddings_dict.keys()]))

tst['modelling']=tst['modelling'].apply(lambda x:[re.sub('[^a-z<>]+', '',i) for i in x])

tst['modelling']=tst['modelling'].apply(lambda x:[i for i in x if len(i)>0 ]) # i not in ['.','\'',','] and

tst['docVec']=tst['modelling'].apply(lambda x: build_doc_vec(x, embeddings_dict, 200))

tst[['txtVec_D'+str(i) for i in range(200)]]=pd.DataFrame(tst.docVec.tolist())

In [50]:
tst['keyword']=tst['keyword'].fillna('unknown', axis=0)

tst['keyword1']=tst['keyword'].apply(lambda x: stemm.stem(re.sub(r'[^a-zA-Z]+', ' ', x)))
tst['keyword1']=tst['keyword1'].apply(lambda x: keywordMapp[x])

tstp=pd.get_dummies(tst[['keyword1']]).drop('keyword1_unknown', axis=1)
tstp=pd.DataFrame(pca.transform(tstp))

tst[tstp.columns]=tstp[tstp.columns]

In [51]:
tst.columns

Index([                    'id',                'keyword',
                     'location',                   'text',
                'processingtxt',                'retweet',
                     'link_cnt',               'tags_cnt',
                 'hashTags_cnt', 'glove_tweet_processing',
       ...
                            105,                      106,
                            107,                      108,
                            109,                      110,
                            111,                      112,
                            113,                      114],
      dtype='object', length=333)

In [62]:
op=tst.drop(['keyword', 'location', 'text', 'processingtxt','keyword1',
            'glove_tweet_processing', 'docVec',
           'glove_tweet_processing1', 'glove_tweet_processing2' ,
            'glove_tweet_processing3', 'glove_tweet_processing4' ,'glove_tweet_processing5' ,
            'modelling',
           ], axis=1)

In [63]:
# op['target']=ann_e.predict(op.drop('id', axis=1))
op['target']=xgb.predict(op.drop(['id'], axis=1))


In [64]:
op['target']=op['target'].apply(lambda x: 1 if x>=0.5 else 0)

In [65]:
op[['id', 'target']].to_csv('output1.csv', index=False)

In [432]:
corpus=[]
for i in df.processingtxt.values:
    corpus.extend(i)

In [433]:
corpus=set(corpus)

In [434]:
len(corpus)

13203

In [435]:
vocab_size=len(corpus)+1

In [413]:
word2idx={}
for idx, wrd in enumerate(corpus):
    word2idx[wrd]=idx

In [231]:
build_doc_vex(['hi', 'i', 'am', 'awesome'])

array([-0.05560975,  0.24205375,  0.60504251, -0.23084063, -0.855195  ,
       -0.00665749, -0.00852025,  0.37282749,  0.19691   , -0.0721488 ,
        0.8910725 ,  0.133813  ,  0.03958751, -0.257634  ,  0.14736501,
       -0.30702724, -0.1375325 ,  0.399735  , -0.491021  ,  0.6050175 ,
        0.23926051,  0.746705  , -0.27301375, -0.23784749,  0.3009825 ,
        0.5598425 ,  0.41278248, -0.214125  ,  0.56112999, -0.02849823,
       -0.4968625 ,  0.56169226,  0.5870525 ,  0.15311675,  0.0922645 ,
       -0.03146001, -0.4577025 ,  0.284877  ,  0.2720425 , -0.57395249,
        0.1637475 , -0.0974975 ,  0.18299501, -0.47561151, -0.4297105 ,
       -0.35641247,  0.35213251, -0.0964485 ,  0.05626175, -1.15160251,
       -0.62553375, -0.00807175,  0.32719   ,  0.354175  , -0.51479748,
       -1.56865998,  0.5683125 ,  0.83094001,  0.857435  ,  0.08458225,
        0.20520849,  0.7345225 , -0.92396498, -0.06143242,  0.36030751,
        0.030791  ,  0.35041749,  0.2934425 , -0.17469751,  0.05

In [226]:
np.zeros(100)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [280]:
len(set(corpus)-(set(embeddings_dict.keys()).intersection(corpus)))

1321

In [414]:
idx2word={v:k for k, v in word2idx.items()}

In [415]:
df['textSequences']=df['processingtxt'].apply(lambda x:[word2idx[i] for i in x])

In [416]:
from keras.preprocessing.sequence import skipgrams

In [436]:
df['3-Grams']=df['processingtxt'].apply(lambda x: skipgrams(x, vocabulary_size=vocab_size, window_size=3 ))

In [437]:
df.head()

,id,keyword,location,text,target,processingtxt,link_cnt,tags_cnt,hashTags_cnt,retweet,textSequences,3-Grams
0,1,unknown,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1,"[our, deed, are, the, reason, of, this, earthquake, may, allah, forgive, u, all]",0,0,1,0,"[11457, 9212, 6710, 11712, 6587, 3307, 10558, 8842, 6751, 4681, 758, 2459, 13047]","([[allah, earthquake], [deed, 4276], [may, 11385], [of, 12533], [reason, this], [all, allah], [earthquake, 9901], [our, 6132], [all, 5351], [earthquake, 2358], [the, reason], [may, 12574], [earthquake, may], [u, all], [are, our], [are, of], [our, the], [are, 1867], [forgive, 1727], [u, allah], [of, 963], [of, 10259], [reason, the], [are, deed], [all, u], [may, allah], [reason, deed], [earthquake, reason], [earthquake, allah], [deed, 2466], [the, this], [forgive, all], [earthquake, 4168], [this, 7389], [are, reason], [are, 10045], [our, 3971], [allah, 11786], [of, reason], [this, the], [may, 10754], [allah, u], [are, the], [forgive, 2571], [are, 1654], [this, 145], [reason, 4235], [allah, this], [reason, 9494], [earthquake, 9194], [all, forgive], [of, 11856], [may, earthquake], [reason, 7353], [forgive, 5544], [u, may], [deed, 5014], [may, 1570], [earthquake, 9292], [forgive, u], [of, may], [the, 5764], [of, this], [allah, forgive], [the, deed], [the, 5051], [of, the], [the, 6215], [earthquake, 6316], [reason, 7283], [this, of], [earthquake, of], [deed, the], [may, of], [this, may], [deed, reason], [may, 3149], [forgive, may], [this, reason], [the, are], [allah, may], [u, 8353], [of, earthquake], [reason, are], [our, are], [deed, our], [u, 8304], [reason, earthquake], [allah, 12993], [the, 7783], [earthquake, forgive], [this, 11484], [earthquake, this], [deed, are], [reason, 8515], [all, 9216], [all, 9730], [forgive, 11983], [allah, all], [are, 8252], ...], [1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, ...])"
1,4,unknown,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]",0,0,0,0,"[1365, 9015, 6299, 11098, 7765, 3521, 1013]","([[canada, sask], [near, 4128], [fire, 1498], [canada, 12497], [sask, 10879], [fire, 2821], [fire, la], [la, 2003], [sask, 10326], [la, 4938], [fire, ronge], [la, near], [la, fire], [fire, 2402], [near, sask], [canada, 3159], [fire, forest], [la, 193], [forest, near], [ronge, canada], [sask, 3347], [near, 8988], [la, 9224], [la, 3999], [la, canada], [near, la], [fire, 7804], [sask, 1339], [canada, 628], [ronge, 12521], [near, fire], [ronge, la], [ronge, fire], [sask, la], [ronge, 11080], [canada, la], [sask, near], [near, ronge], [ronge, 11848], [ronge, near], [near, 3403], [la, forest], [forest, 11992], [sask, canada], [near, forest], [forest, la], [forest, 7390], [la, 5696], [forest, 11774], [forest, fire], [near, 12764], [la, ronge], [ronge, 5277], [la, sask], [ronge, 156], [near, 5987], [fire, near], [canada, ronge], [sask, ronge], [ronge, sask]], [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1])"
2,5,unknown,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1,"[all, resident, asked, to, ishelter, in, place, are, being, notified, by, officer, no, other, evacuation, or, shelter, in, place, order, are, expected]",0,0,0,0,"[13047, 9193, 7046, 6799, 4034, 5624, 1297, 6710, 3669, 6214, 5796, 3796, 9519, 8342, 314, 7555, 7552, 5624, 1297, 11554, 6710, 6622]","([[ishelter, to], [place, being], [or, other], [all, 5777], [place, notified], [officer, 2269], [evacuation, 6245], [or, 5004], [in, asked], [other, 455], [are, 10229], [ishelter, i

In [222]:
df['keyword'].fillna('unknown', axis=0, inplace=True)

In [443]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
tmp=['This','is', 'Siraj', 'and', 'he', 'is', 'worthy']#, 'I', 'am', 'not', 'able', 'to', 'understand', 'the',
tmp=['This is Siraj and he is a good boy', 
    'I hate Siraj for being a good boy']
#     'logic', 'behind', 'skipgrams']
cv.fit_transform(tmp).todense()

matrix([[1, 0, 1, 0, 1, 0, 1, 2, 1, 1],
        [0, 1, 1, 1, 1, 1, 0, 0, 1, 0]], dtype=int64)

In [446]:
cv.get_feature_names()

['and', 'being', 'boy', 'for', 'good', 'hate', 'he', 'is', 'siraj', 'this']

In [25]:
df['text']=df['text'].apply(lambda x:word_tokenize(x))

In [29]:
df['text']=df['text'].apply(lambda x:[i.lower() for i in x if not i in set(stopwords.words('english'))])

In [54]:
df['text']=df['text'].apply(lambda x:[''.join(re.findall('[a-zA-Z]', i)) for i in x if len(''.join(re.findall('[a-zA-Z]', i)))>1])

In [57]:
corpus=[]
for l in df.text:
    corpus.extend(l)

In [59]:
corpus=set(corpus)

In [64]:
word2idx={}
for i, w in enumerate(corpus):
    word2idx[w]=i

In [67]:
len(corpus)

21196

In [68]:
df['txt_sequences']=df['text'].apply(lambda x: [word2idx[i] for i in x])

In [79]:
x=np.zeros((df.shape[0], len(corpus)))
y=df.target

In [80]:
for i, w in enumerate(df.txt_sequences):
    x[i,w]=1

In [32]:
tst['processingtxt']=tst['text'].apply(lambda x: re.sub('\'m', ' am', x))

tst['retweet']=tst['processingtxt'].apply(lambda x: 0 if re.subn('^RT[^:]*:', '', x)[1]==0 else 1)
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.subn('^RT[^:]*:', '', x)[0])

tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('!',  ' ', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('\'s', ' is', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('can\'t', 'can not', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('ain\'t', 'am not', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('shan\'t', 'shall not', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('n\'t', ' not', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('\'ll', ' will', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('\'ve', ' have', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('\'re', ' are', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('w/', 'with ', x))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.sub('\'ll', ' will', x))

tst['processingtxt']=tst['processingtxt'].apply(lambda x: re.split(' |;|,|\n', x))

tst['link_cnt']=tst['processingtxt'].apply(lambda x: len([i for i in x if 'http' in i]))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: [i for i in x if 'http' not in i])

tst['processingtxt']=tst['processingtxt'].apply(lambda x:[k for j in [i.split('/') for i in x] for k in j])
tst['processingtxt']=tst['processingtxt'].apply(lambda x:[k for j in [i.split('.') for i in x] for k in j])

tst['tags_cnt']=tst['processingtxt'].apply(lambda x: len([i for i in x if len(i)>0 and i[0]=='@']))
tst['processingtxt']=tst['processingtxt'].apply(lambda x: ([i for i in x if len(i)>0 and i[0]!='@']))

tst['hashTags_cnt']=tst['processingtxt'].apply(lambda x: len([i for i in x if len(i)>0 and i[0]=='#']))

# tst['processingtxt']=tst['processingtxt'].apply(lambda x: [re.sub('[^a-zA-Z]', '' , i).lower() for i in x])

# tst['processingtxt']=tst['processingtxt'].apply(lambda x: [i for i in x if len(i)>0])
# tst['processingtxt']=tst['processingtxt'].apply(lambda x: [spell_contrac(i) for i in x])
                                              
# tst['processingtxt']=tst['processingtxt'].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])



In [33]:
tst['glove_tweet_processing']=tst['text'].apply(lambda x: tokenize(x))


tst['glove_tweet_processing1']=tst['glove_tweet_processing'].apply(lambda x:[k for j in [tweet_processed_tokenize(i) for i in re.split(' ',x)] for k in j])

tst['glove_tweet_processing2']=tst['glove_tweet_processing1'].apply(lambda x:[k for j in [handling_angulars(i) for i in x] for k in j])
tst['glove_tweet_processing3']=tst['glove_tweet_processing2'].apply(lambda x:[i for i in x if re.search(r'[a-zA-Z<>]+', i)])




In [35]:
# glove_corpus=[]
# for i in df.glove_tweet_processing3.values:
#     glove_corpus.extend(i)
# glove_corpus=set(glove_corpus)

# # extended_embedded_dict=embeddings_dict.copy()

# train_unknown_corpus= (glove_corpus- set(embeddings_dict.keys()))
# train_unknown_mapp={}
# for wrd in train_unknown_corpus:
#     train_unknown_mapp[wrd]=spell_contrac(wrd)
  

tst_glove_corpus=[]
for i in tst.glove_tweet_processing2.values:
    tst_glove_corpus.extend(i)
tst_glove_corpus=set(tst_glove_corpus)

tst_unknown_corpus= (tst_glove_corpus- set(embeddings_dict.keys()))
tst_unknown_mapp={}
for wrd in tst_unknown_corpus:
    tst_unknown_mapp[wrd]=spell_contrac(wrd)
  

badgeofhonour badgeofhonour
pasokon passion
feelingmanly feelingmanly
handplacementgoals handplacementgoals
katayama nakayama
sharecropping sharecropping
epicente epicentre
biltz blitz
multidimensiona multidimensional
sympathyonlf sympathyonlf
nberg berg
kerricktrial kerricktrial
rodder rudder
sittwe little
viab via
newnewnew newnewnew
publicsafetyfirst publicsafetyfirst
crunchysensible crunchysensible
whelen helen
crematoria crematoria
yiayplan yiayplan
roquey roque
livesmatter livesmatter
phmsa phase
superbactor superbactor
emekagift emekagift
lesmills newmills
dogbite doggie
perturbs perturb
ikely likely
multidimen multidimen
hihow how
slownewsday slownewsday
toothsome toothsome
newsmornings newsmornings
pvdrdlj pvdrdlj
nizer nicer
demolishe demolished
companionate companionate
stationright stationright
epilept epilepsy
wfocus focus
anyname anytime
gorlovka gorlovka
myswc music
mindkiller mindkiller
concertphotography concertphotography
sebastianstanisaliveandwell sebastianstanisali

befoooooooore before
 a
nuclearrc nuclear
xshanemichaelsx xshanemichaelsx
salopek slope
sympathising sympathising
ashens athens
politifiact politifiact
writerslife writerslife
mothernaturenetwork mothernaturenetwork
presqu press
onrhpz onrhpz
jsett jett
unfortun unfortun
jojowizphilipp jojowizphilipp
wroug wrong
likkly likely
tradery traders
newsbrokenemergency newsbrokenemergency
salha salha
photographyuk photography
torpedoed torpedoed
willyt willy
krnv kind
harlet hamlet
incubustour incubustour
rapidcity rapidity
whatstheimportantvideo whatstheimportantvideo
kampers hampers
quivk quick
karasuno karaso
elsewere elsewhere
timbering limbering
gallups gallup
wehtwtvw wehtwtvw
scfd scud
hlps helps
kontagora contadora
tahreer three
ngiqy noisy
aqgco ago
kahel karel
ncjfcj ncjfcj
tankerfire tankerfire
kaimai karma
onbeingwith onbeingwith
mynabirds mynabirds
xrwn down
ruebs rubs
hazzel hazel
qaryatayn qaryatayn
netnewsledger netnewsledger
bhill bill
popobawa popobawa
multitudinal multitudi

In [36]:
tst['glove_tweet_processing3']=tst['glove_tweet_processing3'].apply(lambda x:[unknownMap(i, tst_unknown_mapp) for i in x])


In [37]:

tst['modelling']=tst['glove_tweet_processing3'].apply(lambda x:[i for i in x if i in embeddings_dict.keys()])

tst['docVec']=tst['modelling'].apply(lambda x: build_doc_vec(x))

tst[['txtVec_D'+str(i) for i in range(200)]]=pd.DataFrame(tst.docVec.tolist())

In [38]:
tst['keyword']=tst['keyword'].fillna('unknown', axis=0)

tst['keyword']=tst['keyword'].apply(lambda x: stemm.stem(re.sub(r'[^a-zA-Z]+', ' ', x)))

tmp=pd.get_dummies(tst[['keyword']]).drop('keyword_unknown', axis=1)
tst[tmp.columns]=tmp[tmp.columns]

In [102]:
a=[1,2,3,4,5,6,7,8,9,0,99]
q=zip(*[a[i:] for i in range(3)])

In [104]:
[a[i:] for i in range(3)]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 99],
 [2, 3, 4, 5, 6, 7, 8, 9, 0, 99],
 [3, 4, 5, 6, 7, 8, 9, 0, 99]]

In [103]:
list(q)

[(1, 2, 3),
 (2, 3, 4),
 (3, 4, 5),
 (4, 5, 6),
 (5, 6, 7),
 (6, 7, 8),
 (7, 8, 9),
 (8, 9, 0),
 (9, 0, 99)]

In [ ]:
mdl=Sequential()

mdl.add(Embedding(len(corpus), 100, ))

In [290]:
from sklearn.feature_extraction.text import CountVectorizer

In [400]:
tmp=df[['text']].sample(1000)

In [401]:
cv=CountVectorizer()

In [402]:
x=cv.fit_transform(tmp.text.values)

In [404]:
pd.DataFrame(x.todense()).head()

0  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  \
0  0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0    
1  0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0    
2  0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0    
3  0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0    
4  0  0  0  0  0  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0   0    

   21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  \
0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
1  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
2  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
3  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
4  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    

   40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  \
0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
1  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
2  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
3  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
4  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    

   59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  \
0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
1  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
2  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
3  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
4  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    

   78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  \
0  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
1  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
2  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
3  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    
4  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0    

   97  98  99  100  101  102  103  104  105  106  107  108  109  110  111  \
0  0   0   0   0    0    0    0    0    0    0    0    0    0    0    0     
1  0   0   0   0    0    0    0    0    0    0    0    0    0    0    0     
2  0   0   0   0    0    0    0    0    0    0    0    0    0    0    0     
3  0   0   0   0    0    0    0    0    0    0    0    0    0    0    0     
4  0   0   0   0    0    0    0    0    0    0    0    0    0    0    0     

   112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  \
0  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
1  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
2  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
3  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
4  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     

   127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  \
0  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
1  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
2  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
3  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     
4  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0     

   142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  \
0  0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

In [336]:
tmp['txt']=tmp['text'].apply(lambda x: word_tokenize(x))

In [337]:
corpus=[]
for i in tmp['txt'].values:
    corpus.extend(i)
corpus=set(corpus)

In [388]:
word2idx={wrd: i for i,wrd in enumerate(corpus)}
# for i, wrd in enumerate(corpus):
#     word2idx[wrd]=i

In [389]:
word2idx

{'NEWS': 0,
 'Danger': 1,
 'Edwin': 2,
 'Armenians': 3,
 "'worries": 4,
 'disappointing': 5,
 'Breaking': 6,
 '1979': 7,
 "'An": 8,
 '83': 9,
 'DETECTADO': 10,
 'Guaranteed': 11,
 'Emergency': 12,
 'TREES': 13,
 'panicking': 14,
 'May': 15,
 'road': 16,
 '7a-7:30p': 17,
 'awesome': 18,
 '//t.co/7d7VweQ3eS': 19,
 'Forgiving': 20,
 'Christian': 21,
 'D.C.': 22,
 'assisting': 23,
 'anellatulip': 24,
 'legs': 25,
 'Sackville': 26,
 'underway': 27,
 'broken': 28,
 'ahead': 29,
 'Photos': 30,
 'During': 31,
 'Reasons': 32,
 'islands': 33,
 'big': 34,
 'blast': 35,
 'Rivals': 36,
 'Being': 37,
 'tub': 38,
 'badr58': 39,
 "'M": 40,
 'marks': 41,
 'and': 42,
 'Simulation': 43,
 'green': 44,
 'hijack': 45,
 'Yourself': 46,
 'affected': 47,
 'Know': 48,
 'official': 49,
 'BULLETIN': 50,
 'hashtags': 51,
 'Manslaughter': 52,
 'BlueJays': 53,
 'Mayan': 54,
 'Flood': 55,
 'beach': 56,
 'anniversary': 57,
 'snow': 58,
 '//t.co/Wf8iTK2KVx': 59,
 'destroyed': 60,
 'Daily': 61,
 'mY': 62,
 'cyclone': 63

In [385]:
mt=np.zeros((tmp.shape[0], len(corpus)+1))

In [736]:
similar_word={}
for wrd in (set(train_unknown_mapp.values())-set(embeddings_dict.keys())):
    
    mapp=list(set(embeddings_dict.keys()))[0]
    dis=hamming(wrd, mapp)
    for i in list(set(embeddings_dict.keys()))[1:]:
        tmp=hamming(wrd, i)
        if dis>tmp:
            dis=tmp
            mapp=i
        if dis<3:
            break
    print(wrd, i, dis)
    similar_word[wrd]=i

fieldstone akihide 3
nflexpertpicks akihide 8
aintsheperty akihide 6
gravitychat akihide 4
tullamarine akihide 3
pyrotechnic pyrotechnics 1
labourleadership akihide 9
novalismi novalis 2
josephus josephs 2
driverlesscars akihide 4
ripriprip akihide 3
blacklivesmatter akihide 8
owenrbroadhurst akihide 10
airguns birgun 2
cutekitten akihide 4
roskomnadzor akihide 7
bronwynbishop akihide 6
urgentthere akihide 5
championsblackfoot akihide 8
amnisos amassos 2
graywardens akihide 5
bajrangi bajrang 1
thisisfaz thisisfun 2
legionnaire legionnaires 1
teamsurvivors akihide 6
wahpeton akihide 3
clothesless akihide 3
otrametlife akihide 5
sanitised sanitized 1
robertwelch akihide 5
paulhollywood akihide 7
yeyeulala akihide 4
poconorecord akihide 5
dougkessler akihide 5
bigrigradio akihide 4
keepingtheviginaclean akihide 13
stopharper akihide 3
reidlake akihide 3
duststorm akihide 3
icemoon icewood 2
autoinsurance akihide 4
quarterstaff akihide 4
difficultpeople akihide 5
nikoniko ninonino 2
snotg

kororinpa akihide 3
dissuaded dissuade 1
damiengayle akihide 5
magginoodle akihide 5
gamefeed gameweek 2
reg. re 2
infectiousdiseases akihide 8
superintend akihide 3
hendrixonfire akihide 6
scaevola akihide 3
townswomen akihide 4
expertwhiner akihide 5
unscreened akihide 3
salvages salvate 2
oktxduo akihide 3
watersafety akihide 4
presuppose presupposes 1
hubel nuber 2
ibooklove akihide 4
bioterror akihide 4
unfortunemelody akihide 5
grafoscopia akihide 4
comus comam 2
unexercised akihide 3
sevenfigz akihide 3
stowing studing 2
nlccollide akihide 4
rewatchingthepilot akihide 8
liawski akihide 3
insensibly invensible 2
janenelson akihide 4
dumuzid akihide 3
inundation inundacion 1
anthelmintic akihide 5
astrakhan akihide 3
angioplasty akihide 4
kwwkwwk wkwkwwk 2
seasonfrom akihide 4
thereisonlysex akihide 6
hearths heart.i 2
znf auf 2
wantmyabsback akihide 6
hempel bemmel 2
pakpattan akihide 3
ninet nikut 2
skanndtyagi akihide 6
krimsky grimsby 2
atombomb akihide 3
heavydirtysoul akihid

KeyboardInterrupt: 

In [394]:
tmp.reset_index(drop=True, inplace=True)

In [377]:
t=np.array(tmp.txt1.values)

In [386]:
for i in range(t.shape[0]):
    for j in t[i]:
        mt[(i,j)]+=1

In [387]:
mt[(1,t[1])]

array([1., 1., 2., 1., 1., 1., 2., 1., 2., 1., 1., 2., 1., 1.])